# SC-SINDy Quickstart

This notebook demonstrates the basic usage of Structure-Constrained SINDy.

In [ ]:
# Install if needed
# !pip install sc-sindy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sc_sindy import (
    sindy_stls,
    build_library_2d,
    compute_derivatives_finite_diff,
    format_equation,
)
from sc_sindy.systems import VanDerPol

## 1. Generate Data

We'll use the Van der Pol oscillator as an example.

In [ ]:
# Create system and generate trajectory
system = VanDerPol(mu=1.0)
t, X = system.simulate([1.0, 0.0], t_span=(0, 20), dt=0.01)

print(f"Generated {len(t)} data points")
print(f"State shape: {X.shape}")

In [ ]:
# Plot trajectory
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(t, X[:, 0], label='x')
plt.plot(t, X[:, 1], label='y')
plt.xlabel('Time')
plt.legend()
plt.title('Time Series')

plt.subplot(122)
plt.plot(X[:, 0], X[:, 1])
plt.xlabel('x')
plt.ylabel('y')
plt.title('Phase Portrait')
plt.tight_layout()
plt.show()

## 2. Compute Derivatives

In [ ]:
# Compute derivatives
dt = t[1] - t[0]
X_dot = compute_derivatives_finite_diff(X, dt)
print(f"Derivative shape: {X_dot.shape}")

## 3. Build Library and Discover Equations

In [ ]:
# Build polynomial library
Theta, labels = build_library_2d(X)
print(f"Library shape: {Theta.shape}")
print(f"Library terms: {labels}")

In [ ]:
# Run SINDy
xi, iterations = sindy_stls(Theta, X_dot, threshold=0.1)
print(f"Converged in {iterations} iterations")
print(f"Coefficient matrix shape: {xi.shape}")

## 4. Display Results

In [ ]:
# Format and display equations
var_names = ['x', 'y']
for i, var in enumerate(var_names):
    eq = format_equation(xi[i], labels, var_name=f"d{var}/dt")
    print(eq)

In [ ]:
# Compare with ground truth
print("\nTrue equations:")
print("dx/dt = y")
print("dy/dt = mu*(1 - x^2)*y - x")
print(f"where mu = {system.mu}")